In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


# Split dataset

In [ ]:
import pandas as pd

# train_data = pd.read_csv("data/train_data_v5.csv")
# test_data = pd.read_csv("data/test_data_v5.csv")
train_data = pd.read_csv("data/merged_dataset.csv")

In [ ]:
import numpy as np

def split_train_test(data, test_size):
  shuffled = np.random.permutation(len(data))
  num_test = int(test_size*len(data))
  test_index = shuffled[:num_test]
  train_index = shuffled[num_test:]
  return data.iloc[train_index], data.iloc[test_index]

while True:
  train_set, val_set = split_train_test(train_data, 0.2)
  ratio = (train_data.iloc[:, 1:] != 0).sum() / len(train_data)
  train_ratio = (train_set.iloc[:, 1:] != 0).sum() / len(train_set)
  val_ratio = (val_set.iloc[:, 1:] != 0).sum() / len(val_set)
  if np.all(((ratio - val_ratio).abs() < 0.002).values) and np.all(((ratio - train_ratio).abs() < 0.002).values):
    break

In [ ]:
len(train_set), len(val_set)

(3992, 997)

In [ ]:
train_set.to_csv('data/train_data_vv1.csv')
val_set.to_csv('data/test_data_vv1.csv')

# Install packages

In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
!pip install transformers
!pip install vncorenlp
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load datasets

In [ ]:
from datasets import load_dataset

data_files = {'train': "data/train_data_v7.csv", 'test': "data/test_data_v7.csv"}
dataset = load_dataset('csv', data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a64a42590cbc3894/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

# Preprocess data

In [ ]:
# files = open("vietnamese-stopwords.txt", "r")
# STOPWORDS = files.read()
# STOPWORDS = STOPWORDS.split("\n")
# STOPWORDS = set(STOPWORDS)
# STOPWORDS.remove('không')

# def remove_stopwords(text):
#     """custom function to remove the stopwords"""
#     return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [ ]:
import re

dict_look = {'ks':'khách sạn', 
             'KS':'khách sạn',
             'Ks':'khách sạn',
             'ksan':'khách sạn',
             'Ksan':'khách sạn',
             'khach san':'khách sạn',
             'nv':'nhân viên',
             '\n':'. ',
             '\r.':'',
             '\r':'',
             ';':'',
             }

def stand_text(text):
    for k, v in dict_look.items():
      text = text.replace(k, v)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_special_char(text):
    special_character = re.compile("�+")
    return special_character.sub(r'', text)

def remove_punctuation(text):
    punctuation = re.compile("[!#$%&()*+;<=>?@[\]^_`{|}~]")
    return punctuation.sub(r"", text)

def lowercasing(text):
    return text.lower()


In [ ]:
def clean_text(example):
  return {"Review": lowercasing(remove_punctuation(remove_special_char(remove_emoji(stand_text(example["Review"])))))}

def segment(example):
  return {"Segment": " ".join([" ".join(sen) for sen in rdrsegmenter.tokenize(example["Review"])])}

dataset = dataset.map(clean_text)
dataset = dataset.map(segment)

  0%|          | 0/3913 [00:00<?, ?ex/s]

  0%|          | 0/978 [00:00<?, ?ex/s]

  0%|          | 0/3913 [00:00<?, ?ex/s]

  0%|          | 0/978 [00:00<?, ?ex/s]

In [ ]:
import numpy as np

feature = ['giai_tri', 'luu_tru', 'nha_hang', 'an_uong', 'di_chuyen', 'mua_sam']

def tokenize(example):
  return tokenizer(example["Segment"], truncation=True)

def label(example):
  return {'labels_regressor': np.array([example[i] for i in feature]),
          'labels_classifier': np.array([int(example[i] != 0) for i in feature])}

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.map(label)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3913 [00:00<?, ?ex/s]

  0%|          | 0/978 [00:00<?, ?ex/s]

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets = dataset.remove_columns(['Unnamed: 0','Review', 'giai_tri', 'luu_tru', 'nha_hang', 'an_uong', 'di_chuyen', 'mua_sam', 'Segment'])
tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig
class CustomModelSoftmax(nn.Module):
  def __init__(self, checkpoint,):
    super(CustomModelSoftmax, self).__init__()
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768*4, 6)
    self.regressor = nn.Linear(768*4, 30)
    # self.output2 = nn.Linear(96, 6)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    # outputs = self.dropout(outputs_sequence[2])
    outputs = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
    outputs = self.dropout(outputs)
    # outputs = self.output1(outputs[:, 0, :])
    outputs_classifier = self.classifier(outputs)
    outputs_regressor = self.regressor(outputs)
    outputs_regressor = outputs_regressor.reshape(-1, 6, 5)
    # outputs = self.output2(outputs)
    outputs_classifier = nn.Sigmoid()(outputs_classifier)
    # outputs_regressor = nn.Sigmoid()(outputs_regressor)*5
    return outputs_classifier, outputs_regressor

In [ ]:
import torch.nn.functional as F


def loss_classifier(pred_classifier, labels_classifier):
  return nn.BCELoss()(pred_classifier, labels_classifier)


def loss_regressor(pred_regressor, labels_regressor):
  mask = (labels_regressor != 0)
  loss = ((pred_regressor - labels_regressor)**2)[mask].sum() / mask.sum()
  return loss

def loss_softmax(inputs, labels, device):
  mask = (labels != 0)
  # inputs (N, 6, 5)
  n, aspect, rate = inputs.shape
  num = 0
  loss = torch.zeros(labels.shape).to(device)
  for i in range(aspect):
    # print(inputs[:, i, :].shape, labels[:, i].shape)
    label_i = labels[:, i].clone()
    label_i[label_i != 0] -= 1
    label_i = label_i.type(torch.LongTensor).to(device)
    loss[:, i] = nn.CrossEntropyLoss(reduction='none')(inputs[:, i, :], label_i)
  loss = loss[mask].sum() / mask.sum()
  return loss

def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.25,
    gamma: float = 2,
    reduction: str = "none",):

    # p = torch.sigmoid(inputs)
    p = inputs
    ce_loss = F.binary_cross_entropy(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

def bce_loss_weights(inputs, targets, weights):
  ce_loss = F.binary_cross_entropy(inputs, targets, reduction="none")
  weights = targets*(1 / weights.view(1, -1)) + (1 - targets)*(1 / (1 - weights.view(1, -1)))
  loss = ce_loss*weights
  return loss.mean()


def CB_loss(inputs, targets, samples_positive_per_cls, samples_negative_per_cls, no_of_classes=2,loss_type='sigmoid', beta=0.9999, gamma=2):
  # print(targets.shape)
  # samples_positive_per_cls = targets.sum(dim=0) # num_cls
  # samples_negative_per_cls = len(targets) - samples_positive_per_cls # num_cls
  samples_per_cls = torch.concat([samples_positive_per_cls.unsqueeze(-1), samples_negative_per_cls.unsqueeze(-1)], dim=-1) # num_cls, 2
  # print(samples_per_cls.shape)
  effective_num = 1.0 - torch.pow(beta, samples_per_cls) # num_cls, 2
  weights = (1.0 - beta) / effective_num # num_cls, 2
  # print(weights.shape)
  weights = weights / weights.sum(dim=-1).reshape(-1, 1) * no_of_classes # num_cls, 2 
  weights = targets*weights[:, 0] + (1 - targets)*weights[:, 1]

  if loss_type == "focal":
    cb_loss = (sigmoid_focal_loss(inputs, targets)*weights).mean()
  elif loss_type == "sigmoid":
    cb_loss = (F.binary_cross_entropy(inputs,targets, reduction="none")*weights).mean()

  return cb_loss

# Post-processing

In [ ]:
def pred_to_label(outputs_classifier, outputs_regressor):
  result = np.zeros((outputs_classifier.shape[0], 6))
  mask = (outputs_classifier >= 0.5)
  result[mask] = outputs_regressor[mask]
  return result

# Metrics

In [ ]:
class ScalarMetric():
  def __init__(self):
    self.scalar = 0
    self.num = 0
  def update(self, scalar):
    self.scalar += scalar
    self.num += 1
    return self
  def compute(self):
    return self.scalar / self.num
  def reset(self):
    self.scalar = 0
    self.num = 0

class AccuracyMetric():
  def __init__(self):
    self.correct = 0
    self.num = 0
  def update(self, y_pred, y_true):
    self.correct += (y_pred == y_true).sum()
    self.num += len(y_pred)*y_pred.shape[1]
  def compute(self):
    return self.correct / self.num
  def reset(self):
    self.correct = 0
    self.num = 0

def precision(y_pred, y_true):
  true_positive = np.logical_and(y_pred, y_true).sum(axis=0)
  false_positive = np.logical_and(y_pred, np.logical_not(y_true)).sum(axis=0)
  return true_positive / (true_positive + false_positive)

def recall(y_pred, y_true):
  true_positive = np.logical_and(y_pred, y_true).sum(axis=0)
  false_negative = np.logical_and(np.logical_not(y_pred), y_true).sum(axis=0)
  return true_positive / (true_positive + false_negative)

class F1_score():
  def __init__(self):
    self.y_pred = None
    self.y_true = None
  def update(self, y_pred, y_true):
    self.y_pred = np.concatenate([self.y_pred, y_pred], axis=0) if self.y_pred is not None else y_pred
    self.y_true = np.concatenate([self.y_true, y_true], axis=0) if self.y_true is not None else y_true
  def compute(self):
    # print(self.y_pred != 0)
    # print(self.y_true != 0)
    f1_score = np.zeros(self.y_pred.shape[1])
    precision_score = precision(self.y_pred != 0, self.y_true != 0)
    recall_score = recall(self.y_pred != 0, self.y_true != 0)
    mask_precision_score = np.logical_and(precision_score != 0, np.logical_not(np.isnan(precision_score)))
    mask_recall_score = np.logical_and(recall_score != 0, np.logical_not(np.isnan(recall_score)))
    mask = np.logical_and(mask_precision_score, mask_recall_score)
    print("Precision:",precision_score)
    print("Recall", recall_score)
    f1_score[mask] = 2* (precision_score[mask] * recall_score[mask]) / (precision_score[mask] + recall_score[mask])
    return f1_score

class R2_score():
  def __init__(self):
    self.y_pred = None
    self.y_true = None
  def update(self, y_pred, y_true):
    self.y_pred = np.concatenate([self.y_pred, y_pred], axis=0) if self.y_pred is not None else y_pred
    self.y_true = np.concatenate([self.y_true, y_true], axis=0) if self.y_true is not None else y_true
  def compute(self):
    mask = np.logical_and(self.y_pred !=0, self.y_true != 0)
    rss = (((self.y_pred - self.y_true)**2)*mask).sum(axis=0) 
    k = (mask*16).sum(axis=0)
    r2_score = np.ones(rss.shape[0])
    mask2 = (k != 0)
    r2_score[mask2] = 1 - rss[mask2]/k[mask2]
    return r2_score

In [ ]:
model = CustomModelSoftmax("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

CustomModelSoftmax(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 20
num_training_steps = num_epochs*len(train_dataloader)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# Training

In [ ]:
from tqdm.auto import tqdm

pb_train = tqdm(range(num_training_steps))
pb_test = tqdm(range(num_epochs*len(test_dataloader)))
best_score = -1

for epoch in range(num_epochs):
  train_loss = 0
  val_loss = 0
  model.train()
  for batch in train_dataloader:
    inputs = {'input_ids': batch['input_ids'].to(device),
             'attention_mask': batch['attention_mask'].to(device)}
    outputs_classifier, outputs_regressor = model(**inputs)
    # loss = nn.MSELoss()(outputs, batch['labels'].to(device).float())
    # loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
    loss1 = sigmoid_focal_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), alpha=-1, gamma=1,reduction='mean')
    # loss1 = bce_loss_weights(outputs_classifier, batch['labels_classifier'].to(device).float(), weights.to(device))
    # loss1 = CB_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), num_positive.to(device), num_negative.to(device))
    loss2 = loss_softmax(outputs_regressor, batch['labels_regressor'].to(device).float(), device)
    loss = 10*loss1 + loss2
    optimizer.zero_grad()
    # print(loss.item())
    loss.backward()
    optimizer.step()       
    lr_scheduler.step()
    pb_train.update(1)
    pb_train.set_postfix(loss_classifier=loss1.item(),loss_regressor=loss2.item(),loss=loss.item())
    train_loss += loss.item() / len(train_dataloader)
  print("Train Loss:", train_loss)
  model.eval()
  val_loss = ScalarMetric()
  val_loss_classifier = ScalarMetric()
  val_loss_regressor = ScalarMetric()
  val_acc = AccuracyMetric()
  val_f1_score = F1_score()
  val_r2_score = R2_score()
  num = 0
  correct = 0
  result = None
  model.eval()
  for batch in test_dataloader:
    inputs = {'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)}
    with torch.no_grad():
      outputs_classifier, outputs_regressor = model(**inputs)
      # outputs = nn.Sigmoid()(outputs)
      # loss = nn.BCELoss()(outputs, batch['labels'].to(device).float())
      loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
      loss2 = loss_softmax(outputs_regressor, batch['labels_regressor'].to(device).float(), device)
      loss = loss1 + loss2
      outputs_classifier = outputs_classifier.cpu().numpy()
      outputs_regressor = outputs_regressor.cpu().numpy()
      outputs_regressor = outputs_regressor.argmax(axis=-1) + 1
      y_true = batch['labels_regressor'].numpy()
      outputs = pred_to_label(outputs_classifier, outputs_regressor)
      # y_true = prob_to_label(y_true)
      # print(outputs)
      result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
      val_loss_classifier.update(loss1.item())
      val_loss_regressor.update(loss2.item())
      val_loss.update(loss.item())
      val_acc.update(np.round(outputs), y_true)
      val_f1_score.update(np.round(outputs), y_true)
      val_r2_score.update(np.round(outputs), y_true)
      pb_test.update(1)
  f1_score = val_f1_score.compute()
  r2_score = val_r2_score.compute()
  final_score = (f1_score * r2_score).sum()*1/6
  if final_score > best_score:
    best_score = final_score
    torch.save(model.state_dict(), "weights/model_softmax_vv1.pt")
  print("Test Loss:", val_loss.compute(), "Loss Classifier:", val_loss_classifier.compute(), "Loss Regressor:", val_loss_regressor.compute())
  print("Acc", val_acc.compute())
  print("F1_score", f1_score)
  print("R2_score", r2_score)
  print("Final_score", final_score)
  print("Best_score", best_score)

  0%|          | 0/2460 [00:00<?, ?it/s]

  0%|          | 0/620 [00:00<?, ?it/s]

Train Loss: 2.631609150064669
Precision: [0.90361446 0.95798319 0.90434783 0.86582278 0.98412698 0.93197279]
Recall [0.78125    0.77027027 0.85950413 0.92682927 0.84931507 0.94482759]
Test Loss: 1.245275943509994 Loss Classifier: 0.19292995862422452 Loss Regressor: 1.0523459834437217
Acc 0.8824130879345603
F1_score [0.83798883 0.85393258 0.88135593 0.89528796 0.91176471 0.93835616]
R2_score [0.93       0.90844298 0.93449519 0.93366228 0.91532258 0.90875912]
Final_score 0.816982783396723
Best_score 0.816982783396723
Train Loss: 1.5338449090476924
Precision: [0.93292683 0.93129771 0.90265487 0.9301676  0.91275168 0.91836735]
Recall [0.796875   0.82432432 0.84297521 0.90243902 0.93150685 0.93103448]
Test Loss: 1.0489212197642173 Loss Classifier: 0.1538788378238678 Loss Regressor: 0.8950423925153671
Acc 0.8962167689161554
F1_score [0.85955056 0.87455197 0.87179487 0.91609354 0.9220339  0.92465753]
R2_score [0.96364379 0.96516393 0.95772059 0.96602853 0.94163603 0.95277778]
Final_score 0.85